In [1]:
%load_ext autoreload
%autoreload 2

import cv2 as cv
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from qdanalysis.preprocessing.strokedecomposition import simple_stroke_segment
import pathlib

In [2]:
#to make generated figures easier to view
mpl.rcParams['figure.dpi']=300

In [3]:
#path_name = '../datasets/CERUG'
path_name = './CERUG'
path = pathlib.Path(path_name)
files = [fp for fp in path.iterdir()]
files

[PosixPath('CERUG/Writer0909_03-01.ppm'),
 PosixPath('CERUG/Writer0202_03-02.ppm'),
 PosixPath('CERUG/Writer0202_01.ppm')]

In [4]:
from skimage.morphology import skeletonize
#associative arrays, temp
writers = []
pages = []
page_part = []
strokes = []

for file in path.iterdir():
    filename = file.stem
    print(filename)

    writer_no, page_no, page_part_no = re.search(r'(\d*)_(\d*)-?(\d*)?', filename).groups()
    writers.append(writer_no)
    pages.append(page_no)
    page_part.append(page_part_no)

    #open image and process strokes
    kernel_size = (5, 5)
    image = cv.imread(str(file), cv.IMREAD_GRAYSCALE)
    image = cv.GaussianBlur(image, kernel_size, 1.5)
    outfile = './output/' + filename  + '.png'
    cv.imwrite(outfile, image)

    temp = cv.threshold(image, 0, 1, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]
    print(temp)
    temp = temp*255
    outfile = './output/' + filename + "_skele_" + '.png'
    
    cv.imwrite(outfile, temp)

    strokes.append(simple_stroke_segment(image))

Writer0909_03-01
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Writer0202_03-02
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Writer0202_01
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [5]:
writer_1_stroke_sz = map(lambda x: x.shape, strokes[0])
np.array(list(writer_1_stroke_sz)).max(axis=0)

array([469,  39])

In [48]:
import pandas as pd
stroke_df = pd.DataFrame(columns=['writer', 'image'])
for idx, key in enumerate(writers):
    doc_strokes = {"writer": [writers[idx]]*len(strokes[idx]), "image": strokes[idx]}
    
    stroke_df = pd.concat([stroke_df, pd.DataFrame(doc_strokes)], axis=0)

In [62]:
stroke_df['image'][0]

0    [[False, False, False, False, False, False, Tr...
0    [[False, False, False, False, False, False, Fa...
0    [[False, False, True, True, True, True, False,...
Name: image, dtype: object

In [32]:
list(writers[0])

['0', '9', '0', '9']

In [34]:
d = {"writer": [writers[0]]*len(strokes[0]), "image": strokes[0]}
pd.DataFrame(d).shape

(960, 2)